# 事前準備

## Google Drive

Google Driveをマウント

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 地域コード

GoogleDriveから地域一覧を取得する

In [40]:
import json

with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)

print(arealist)

[{'index': 0, 'prefCode': 1, 'prefName': '北海道', 'cityCode': '01100', 'cityName': '札幌市', 'bigCityFlag': '2', 'areaCode': '01100', 'areaName': '札幌市', 'governmentType': 'city'}, {'index': 1, 'prefCode': 1, 'prefName': '北海道', 'cityCode': '01101', 'cityName': '札幌市中央区', 'bigCityFlag': '1', 'areaCode': '01101', 'areaName': '札幌市中央区', 'governmentType': 'city'}, {'index': 2, 'prefCode': 1, 'prefName': '北海道', 'cityCode': '01102', 'cityName': '札幌市北区', 'bigCityFlag': '1', 'areaCode': '01102', 'areaName': '札幌市北区', 'governmentType': 'city'}, {'index': 3, 'prefCode': 1, 'prefName': '北海道', 'cityCode': '01103', 'cityName': '札幌市東区', 'bigCityFlag': '1', 'areaCode': '01103', 'areaName': '札幌市東区', 'governmentType': 'city'}, {'index': 4, 'prefCode': 1, 'prefName': '北海道', 'cityCode': '01104', 'cityName': '札幌市白石区', 'bigCityFlag': '1', 'areaCode': '01104', 'areaName': '札幌市白石区', 'governmentType': 'city'}, {'index': 5, 'prefCode': 1, 'prefName': '北海道', 'cityCode': '01105', 'cityName': '札幌市豊平区', 'bigCityFlag': '1',

# 関数テスト

estat-APIのパラメータ作成

In [42]:
params = {
    'statsDataId':'0000020201',
    'cdCat01':'A1101',
    }

## estat-APIのレスポンスを取得

estat-APIのレスポンス（JSON）を取得する関数

In [43]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

import urllib.parse
import urllib.request
import json

"""
estat-APIのレスポンスを取得する関数
"""

def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

In [44]:
res = get_estat_response(params)
print(res)

{'GET_STATS_DATA': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2022-09-30T12:31:32.791+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0000020201', 'NARROWING_COND': {'CODE_CAT01_SELECT': 'A1101'}, 'DATA_FORMAT': 'J', 'START_POSITION': 1, 'METAGET_FLG': 'Y'}, 'STATISTICAL_DATA': {'RESULT_INF': {'TOTAL_NUMBER': 15313, 'FROM_NUMBER': 1, 'TO_NUMBER': 15313}, 'TABLE_INF': {'@id': '0000020201', 'STAT_NAME': {'@code': '00200502', '$': '社会・人口統計体系'}, 'GOV_ORG': {'@code': '00200', '$': '総務省'}, 'STATISTICS_NAME': '市区町村データ 基礎データ（廃置分合処理済）', 'TITLE': {'@no': '0000020201', '$': 'Ａ\u3000人口・世帯'}, 'CYCLE': '年度次', 'SURVEY_DATE': 0, 'OPEN_DATE': '2022-06-21', 'SMALL_AREA': 0, 'MAIN_CATEGORY': {'@code': '99', '$': 'その他'}, 'SUB_CATEGORY': {'@code': '99', '$': 'その他'}, 'OVERALL_TOTAL_NUMBER': 1567404, 'UPDATED_DATE': '2022-06-21', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '市区町村データ', 'TABULATION_SUB_CATEGORY1': '基礎データ（廃置分合処理済）'}, 'TITLE_SPEC': {'TABLE_NAME': 'Ａ\u3000人口・世帯'}}, 'CLA

## JSONをDataFrameに変換

estat-APIのレスポンスをDataFrameに変換する

In [45]:
import pandas as pd

"""
estat-APIのレスポンスをDataFrameに変換する
"""

def conv_estat_response_to_dataframe(response):

  # VALUEをDataFrameに変換
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  df = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df = pd.merge(df, df_class, on=key_name, how='outer')

  # 統計情報を追加
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']
  df['statsDataId'] = TABLE_INF['@id']
  df['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df

In [46]:
df1 = conv_estat_response_to_dataframe(res)
df1

,@tab,@cat01,@area,@time,@unit,$,@tab_name,@cat01_name,@area_name,@time_name,statsDataId,statsDataName
0,00001,A1101,01100,1980100000,人,1401757,観測値,A1101_総人口,北海道 札幌市,1980年度,0000020201,社会・人口統計体系
1,00001,A1101,01101,1980100000,人,181806,観測値,A1101_総人口,北海道 札幌市 中央区,1980年度,0000020201,社会・人口統計体系
2,00001,A1101,01102,1980100000,人,195370,観測値,A1101_総人口,北海道 札幌市 北区,1980年度,0000020201,社会・人口統計体系
3,00001,A1101,01103,1980100000,人,213310,観測値,A1101_総人口,北海道 札幌市 東区,1980年度,0000020201,社会・人口統計体系
4,00001,A1101,01104,1980100000,人,228061,観測値,A1101_総人口,北海道 札幌市 白石区,1980年度,0000020201,社会・人口統計体系
...,...,...,...,...,...,...,...,...,...,...,...,...
15341,NaN,NaN,NaN,2016100000,NaN,NaN,NaN,NaN,NaN,2016年度,0000020201,社会・人口統計体系
15342,NaN,NaN,NaN,2017100000,NaN,NaN,NaN,NaN,NaN,2017年度,0000020201,社会・人口統計体系
15343,NaN,NaN,NaN,2018100000,NaN,NaN,NaN,NaN,NaN,2018年度,0000020201,社会・人口統計体系
15344,NaN,NaN,NaN,2019100000,NaN,NaN,NaN,NaN,NaN,2019年度,0000020201,社会・人口統計体系


## DataFrameを整形

In [72]:
"""
DataFrameを整形する
"""

def  format_estat_dataframe(df_arg):
  # 必要な列だけ抽出
  df =  df_arg[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','$','@unit']]

  # 列名の変更
  columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','$':'value','@unit':'unit'}
  df = df.rename(columns=columns)

  # 欠損データ削除
  df = df.dropna(subset=['categoryCode'])

  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df['timeCode'] = df.apply(lambda x: x['timeCode'][:4], 1)
  
  # 地域情報をマージ
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)
    df_arealist = pd.json_normalize(arealist)  
  df = pd.merge(df, df_arealist, on='areaCode', how='right')


  df = df.dropna(subset=['categoryCode'])

  # '-'を'0'に置換
  df.replace({'value': {'-': '0'}})

  return df

In [73]:
df2 = format_estat_dataframe(df1)
df2

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType
0,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,01100,1401757,人,0,1,北海道,01100,札幌市,2,札幌市,city
1,0000020201,社会・人口統計体系,A1101,総人口,1985,1985年度,01100,1542979,人,0,1,北海道,01100,札幌市,2,札幌市,city
2,0000020201,社会・人口統計体系,A1101,総人口,1990,1990年度,01100,1671742,人,0,1,北海道,01100,札幌市,2,札幌市,city
3,0000020201,社会・人口統計体系,A1101,総人口,1995,1995年度,01100,1757025,人,0,1,北海道,01100,札幌市,2,札幌市,city
4,0000020201,社会・人口統計体系,A1101,総人口,2000,2000年度,01100,1822368,人,0,1,北海道,01100,札幌市,2,札幌市,city
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15306,0000020201,社会・人口統計体系,A1101,総人口,1995,1995年度,47382,1801,人,1921,47,沖縄県,47382,与那国町,0,与那国町,city
15307,0000020201,社会・人口統計体系,A1101,総人口,2000,2000年度,47382,1852,人,1921,47,沖縄県,47382,与那国町,0,与那国町,city
15308,0000020201,社会・人口統計体系,A1101,総人口,2005,2005年度,47382,1796,人,1921,47,沖縄県,47382,与那国町,0,与那国町,city
15309,0000020201,社会・人口統計体系,A1101,総人口,2010,2010年度,47382,1657,人,1921,47,沖縄県,47382,与那国町,0,与那国町,city


## ランキングを付与

In [ ]:
def format_estat_dataframe_withrank(df_arg):

  # 年度リストを作成
  times = df_arg['timeCode'].tolist()
  times = list(set(times))

  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  # 年度毎に順位を付与
  for time in times:

    # 単年度のdataを抽出
    df_time = df_arg.copy()[df_arg['timeCode'] == time]

    # valueを数値に変換して、'rank'を付与
    df_time = df_time.astype({'value': float})
    df_time['rank'] = df_time.rank(ascending=False)['value'].astype(int).astype(str)

    # 結合
    df_res = pd.concat([df_res, df_time])
    df_res = df_res.astype({'value': str})

  return df_res

In [ ]:
df3 = format_estat_dataframe_withrank(df2)
df3

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType,rank
299,0000020201,社会・人口統計体系,A1101,総人口,2010,2010年度,28100,1544200.0,人,1296,28,兵庫県,28100,神戸市,2,神戸市,city,1
300,0000020201,社会・人口統計体系,A1101,総人口,2010,2010年度,28101,210408.0,人,1297,28,兵庫県,28101,神戸市東灘区,1,神戸市東灘区,city,11
301,0000020201,社会・人口統計体系,A1101,総人口,2010,2010年度,28102,133451.0,人,1298,28,兵庫県,28102,神戸市灘区,1,神戸市灘区,city,15
302,0000020201,社会・人口統計体系,A1101,総人口,2010,2010年度,28105,108304.0,人,1299,28,兵庫県,28105,神戸市兵庫区,1,神戸市兵庫区,city,18
303,0000020201,社会・人口統計体系,A1101,総人口,2010,2010年度,28106,101624.0,人,1300,28,兵庫県,28106,神戸市長田区,1,神戸市長田区,city,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,0000020201,社会・人口統計体系,A1101,総人口,2000,2000年度,28464,31960.0,人,1341,28,兵庫県,28464,太子町,0,太子町,city,40
245,0000020201,社会・人口統計体系,A1101,総人口,2000,2000年度,28481,18419.0,人,1342,28,兵庫県,28481,上郡町,0,上郡町,city,48
246,0000020201,社会・人口統計体系,A1101,総人口,2000,2000年度,28501,22337.0,人,1343,28,兵庫県,28501,佐用町,0,佐用町,city,45
247,0000020201,社会・人口統計体系,A1101,総人口,2000,2000年度,28585,23271.0,人,1344,28,兵庫県,28585,香美町,0,香美町,city,44


# まとめ

結果をまとめる

In [93]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

import urllib.parse
import urllib.request
import json
import pandas as pd

"""
estat-APIのレスポンスを整形してDataFrameに変換
"""

def get_estat_dataframe(params):
  res = get_estat_response(params)
  df_res = conv_estat_response_to_dataframe(res)
  df_res = format_estat_dataframe(df_res)

  return df_res

"""
estat-APIのレスポンスを取得する関数
"""

def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))


"""
estat-APIのレスポンスをDataFrameに変換する
"""

def conv_estat_response_to_dataframe(response):

  # VALUEをDataFrameに変換
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  df = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df = pd.merge(df, df_class, on=key_name, how='outer')

  # 統計情報を追加
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']
  df['statsDataId'] = TABLE_INF['@id']
  df['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df

"""
DataFrameを整形する
"""

def  format_estat_dataframe(df_arg):

  # 列を抽出して名前変更
  df =  df_arg[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','$','@unit']]
  df = df.rename(columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','$':'value','@unit':'unit'})

  # 欠損データ削除
  df = df.dropna(subset=['categoryCode'])

  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df['timeCode'] = df['timeCode'].astype(str)
  df['timeCode'] = df.apply(lambda x: x['timeCode'][:4], 1)
  
  # 地域情報をマージ
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)
    df_arealist = pd.json_normalize(arealist)  
  df = pd.merge(df, df_arealist, on='areaCode', how='right')


  df = df.dropna(subset=['categoryCode'])

  # '-'を'0'に置換
  df.replace({'value': {'-': '0'}})

  return df

# テスト

## 市区町村の総人口

In [82]:
params = {
    'statsDataId':'0000020208',
    'cdCat01':'H5609,H560901,H560902,H560903,H560904,H5610',
    }

In [95]:
df_population_city = get_estat_dataframe(params)
df_population_city

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType
0,0000020208,社会・人口統計体系,H5609,ごみ総排出量（総量）,1999,1999年度,01100,965934,ｔ,0,1,北海道,01100,札幌市,2,札幌市,city
1,0000020208,社会・人口統計体系,H5609,ごみ総排出量（総量）,2000,2000年度,01100,976023,ｔ,0,1,北海道,01100,札幌市,2,札幌市,city
2,0000020208,社会・人口統計体系,H5609,ごみ総排出量（総量）,2001,2001年度,01100,983831,ｔ,0,1,北海道,01100,札幌市,2,札幌市,city
3,0000020208,社会・人口統計体系,H5609,ごみ総排出量（総量）,2002,2002年度,01100,948548,ｔ,0,1,北海道,01100,札幌市,2,札幌市,city
4,0000020208,社会・人口統計体系,H5609,ごみ総排出量（総量）,2003,2003年度,01100,945398,ｔ,0,1,北海道,01100,札幌市,2,札幌市,city
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99950,0000020208,社会・人口統計体系,H560901,ごみ総排出量（計画収集量）,2018,2018年度,47382,695,ｔ,1921,47,沖縄県,47382,与那国町,0,与那国町,city
99951,0000020208,社会・人口統計体系,H560902,ごみ総排出量（直接搬入量）,2018,2018年度,47382,74,ｔ,1921,47,沖縄県,47382,与那国町,0,与那国町,city
99952,0000020208,社会・人口統計体系,H5609,ごみ総排出量（総量）,2019,2019年度,47382,750,ｔ,1921,47,沖縄県,47382,与那国町,0,与那国町,city
99953,0000020208,社会・人口統計体系,H560901,ごみ総排出量（計画収集量）,2019,2019年度,47382,668,ｔ,1921,47,沖縄県,47382,与那国町,0,与那国町,city


## 都道府県の総人口

In [80]:
params = {
    'statsDataId':'0000010101',
    'cdCat01':'A1101',
    }

In [81]:
df_population_prefecture = get_estat_dataframe(params)
df_population_prefecture

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType
1922,0000010101,社会・人口統計体系,A1101,総人口,1975,1975年度,01000,5338206,人,0,1,北海道,None,None,None,北海道,prefecture
1923,0000010101,社会・人口統計体系,A1101,総人口,1976,1976年度,01000,5395000,人,0,1,北海道,None,None,None,北海道,prefecture
1924,0000010101,社会・人口統計体系,A1101,総人口,1977,1977年度,01000,5443000,人,0,1,北海道,None,None,None,北海道,prefecture
1925,0000010101,社会・人口統計体系,A1101,総人口,1978,1978年度,01000,5490000,人,0,1,北海道,None,None,None,北海道,prefecture
1926,0000010101,社会・人口統計体系,A1101,総人口,1979,1979年度,01000,5535000,人,0,1,北海道,None,None,None,北海道,prefecture
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4032,0000010101,社会・人口統計体系,A1101,総人口,2015,2015年度,47000,1433566,人,46,47,沖縄県,None,None,None,沖縄県,prefecture
4033,0000010101,社会・人口統計体系,A1101,総人口,2016,2016年度,47000,1439000,人,46,47,沖縄県,None,None,None,沖縄県,prefecture
4034,0000010101,社会・人口統計体系,A1101,総人口,2017,2017年度,47000,1443000,人,46,47,沖縄県,None,None,None,沖縄県,prefecture
4035,0000010101,社会・人口統計体系,A1101,総人口,2018,2018年度,47000,1448000,人,46,47,沖縄県,None,None,None,沖縄県,prefecture
